# All Searching Strategy

## 검색 전략 진화

| 세대 | 방법 | 장점 | 단점 | 프로덕션 사용 |
|------|------|------|------|---------------|
| 1세대 | BM25 (Sparse) | 빠름, 키워드 정확 | 의미 이해 부족 | 단독 사용 비추천 |
| 2세대 | Dense (Bi-encoder) | 의미 이해 우수 | 키워드 약함 | 널리 사용 |
| 3세대 | Hybrid (RRF, Convex Combination) | 두 장점 결합 | 여전히 Keyword 쪽에 치우친 한계 | `현재 구현된 대부분의 RAG 시스템의 표준` |
| **4세대** | **ColBERT(Multi-Vector)** (Late Interaction) | Token-level 정밀도 | 저장 공간 큼 | Computing Cost 고려 필요, 한국어 모델 부족 |
| **5세대** | **SPLADE** (Learned Sparse) | Sparse + 의미 이해 | 차원 높음 (50K) | 한국어 모델 부족 |

---

## 1. ColBERT (Contextualized Late Interaction over BERT)

### 개념

**ColBERT**는 **Late Interaction** 방식의 검색입니다:

**3가지 검색 방식 비교**:

1. **Bi-encoder (Dense)**: 문서/쿼리를 각각 1개 벡터로 압축
   ```
   Query → [단일 벡터]
   Doc → [단일 벡터]
   유사도: 코사인 유사도
   ```

2. **Cross-encoder (Reranker)**: 문서+쿼리를 함께 처리
   ```
   [Query + Doc] → BERT → 점수
   장점: 매우 정확
   단점: 느림 (모든 문서마다 BERT 실행)
   ```

3. **ColBERT (Late Interaction)**: 토큰별로 비교
   ```
   Query → [토큰1 벡터, 토큰2 벡터, ...]
   Doc → [토큰1 벡터, 토큰2 벡터, ...]
   유사도: MaxSim(query 토큰, doc 토큰들)
   ```

**핵심 아이디어**:
- 쿼리 토큰 각각이 문서의 어떤 토큰과 가장 유사한지 계산
- **Token-level 정밀도** → 더 정확한 매칭

**장점**:
- Cross-encoder 수준의 정확도
- Bi-encoder 수준의 속도 (사전 계산 가능)
- **다국어 Cross-lingual 검색** 우수 (LFM2-ColBERT-350M)

**단점**:
- 저장 공간 많이 필요 (모든 토큰 벡터 저장)
- 구현 복잡도 높음

### 참고 자료
- 최신 모델: [LFM2-ColBERT-350M](https://www.liquid.ai/blog/lfm2-colbert-350m-one-model-to-embed-them-all)
- 논문: ["ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT" (Khattab & Zaharia, 2020)](https://arxiv.org/abs/2004.12832)

## ColBERT Late Interaction

### ColBERT 동작 원리

```python
# 1. 문서 인덱싱 시
def encode_document(doc_text, colbert_model):
    # 문서를 토큰별로 임베딩
    tokens = tokenize(doc_text)  # ["벡터", "데이터베이스", "검색", ...]
    token_embeddings = []
    
    for token in tokens:
        embedding = colbert_model.encode_token(token)  # 128-dim 벡터
        token_embeddings.append(embedding)
    
    # 결과: [토큰1 벡터(128-dim), 토큰2 벡터(128-dim), ...]
    return token_embeddings

# 2. 검색 시
def search_colbert(query_text, indexed_docs):
    # 쿼리를 토큰별로 임베딩
    query_tokens = tokenize(query_text)
    query_embeddings = [colbert_model.encode_token(t) for t in query_tokens]
    
    # 각 문서에 대해 MaxSim 계산
    for doc_id, doc_token_embeddings in indexed_docs:
        score = 0
        # 쿼리 토큰 각각에 대해
        for q_emb in query_embeddings:
            # 문서의 모든 토큰과 유사도 계산 → 최대값 선택
            max_sim = max([cosine_similarity(q_emb, d_emb) 
                          for d_emb in doc_token_embeddings])
            score += max_sim
        
        # 최종 점수: 모든 쿼리 토큰의 MaxSim 합
        doc_score = score / len(query_embeddings)
```

### 왜 ColBERT가 더 정확한가?

**예시**: Query = "RAG 시스템", Doc = "검색 증강 생성 (Retrieval Augmented Generation) 시스템"

**Bi-encoder (Dense)**:
```
Query 전체 → [단일 벡터]
Doc 전체 → [단일 벡터]
유사도: 0.75 (전체적으로 유사)
```

**ColBERT**:
```
"RAG" 토큰 → 문서의 "Retrieval", "Augmented", "Generation" 중 최고 유사도
"시스템" 토큰 → 문서의 "시스템"과 정확히 매칭
결과: 더 정밀한 토큰 레벨 매칭 → 0.92
```


## 2. SPLADE (SParse Lexical AnD Expansion)

### 개념

**SPLADE**는 학습된 Sparse 임베딩입니다:
- 기존 BM25: 단순 토큰 빈도 기반
- **SPLADE**: BERT 기반으로 **의미적으로 관련된 토큰에도 가중치 부여**

**핵심 차이**:
```
Query: "RAG 시스템"

BM25 토큰: ["RAG", "시스템"]
SPLADE 토큰: ["RAG", "시스템", "검색", "벡터", "임베딩", "LLM", "AI"] ← 확장!
```

**장점**:
- BM25처럼 빠른 검색 (Inverted Index 사용)
- Dense처럼 의미 이해 (BERT 학습)
- **Query Expansion 자동화** (관련 토큰 자동 추가)

**단점**:
- 차원 매우 높음 (50K) → 메모리 사용량 ↑
- 학습 비용 높음 (BERT fine-tuning)

### 참고 자료
- 한국어 SPLADE 모델: [yjoonjang/splade-ko-v1.0](https://huggingface.co/yjoonjang/splade-ko-v1.0)
- 참고 논문: ["SPLADE v2: Sparse Lexical and Expansion Model for Information Retrieval (Formal et al., 2022)"](https://arxiv.org/abs/2109.10086)

---

## 3. Matryoshka Embeddings 이해(Multi-Vector Search 에 대한 이해와 같음)

### 개념

```
전통적 임베딩: 1536차원 (고정)
Matryoshka: 32, 64, 128, 256, 512, 1024, 1536차원, ... (자유롭게 선택 가능!)
```

**핵심 아이디어**:
- 첫 N 차원만 사용해도 성능 유지
- 예: 처음 256차원만 사용 → 메모리 6배 절감, 성능 ~95% 유지

**사용 사례**:
1. **1차 검색**: 256차원으로 빠르게 후보 100개 검색
2. **2차 Reranking**: 1536차원 full로 정밀 재순위화

### 참고 자료
- [Matryoshka Adaptor 설명](https://jypthemiracle.notion.site/Matryoshka-Adaptor-29aff228a1808010acf8dbefd6580134)
- 논문: ["Matryoshka Representation Learning" (Kusupati et al., 2022)](https://arxiv.org/abs/2205.13147)

### 핵심 아이디어

**Matryoshka Embeddings**는 임베딩의 처음 N 차원만 사용해도 성능을 유지하는 기법으로, **차원의 축소가 가능한 임베딩**입니다.  
일반 임베딩은 모든 차원이 동등하게 중요하지만, Matryoshka는 **앞쪽 차원에 더 많은 정보를 압축**합니다:

```
일반 임베딩 (1536차원):
[0.1, 0.3, -0.2, ..., 0.05, -0.01] ← 모든 차원 필요

Matryoshka (256차원):
[0.8, 0.7, 0.6, ..., 0.01, 0.00] ← 앞 256개만 사용해도 95% 성능!
```

### 사용 전략

**2단계 검색**:
1. **1차**: 256차원으로 빠르게 1000개 검색 (메모리 6배 절약)
2. **2차**: 1536차원 full로 Top 100 재순위화

**모바일/엣지**:
- 64차원만 사용 → 메모리 24배 절약
- 성능 ~85% 유지

---


# 최종 종합: 모든 검색 전략 비교

## 검색 전략 완전 가이드

### 전체 비교 테이블

| 전략 | 정확도 | 속도 | 메모리 | 구현 난이도 | 한국어 | 비용 | 프로덕션 사용 |
|------|--------|------|--------|-------------|--------|------|---------------|
| **BM25 (기본)** | ** | ⚡⚡⚡ | OOO | 쉬움 | ⚠️ | 무료 | 단독 비추천(Kiwi 형태소 분석기 결합 필요) |
| **Kiwi+BM25** | *** | ⚡⚡⚡ | OOO | 쉬움 | OOO | 무료 | Sparse 기본 |
| **SPLADE** | **** | ⚡⚡ | O | 중간 | OO | 무료 | Sparse Embedding |
| **Dense (Bi-encoder)** | **** | ⚡⚡ | OO | 쉬움 | OO | API/로컬 | 프로덕션 표준 |
| **ColBERT** | ***** | ⚡ | O | 어려움 | OO | 로컬 | 고품질 필수 |
| **RRF Hybrid** | **** | ⚡ | OO | 중간 | OOO | API/로컬 | **현재 Best** |

---

## 한국어 최적화 스택

```python
# 1. Embedding: Qwen3-4B via TEI (무료, 빠름)
embedder = HuggingFaceHubEmbeddings(model="http://localhost:8080")

# 2. Sparse: SPLADE-ko (한국어 Query Expansion)
splade = SentenceTransformer('yjoonjang/splade-ko-v1.0')

# 3. Tokenizer: Kiwi 형태소 분석기 (사용자 사전)
kiwi_retriever = KiwiBM25Retriever()

# 4. Hybrid: RRF (Dense 0.5 + SPLADE 0.5) 비중은 조절하면 됩니다.
results = reciprocal_rank_fusion(
    dense_results, 
    splade_results,
    dense_weight=0.5, 
    sparse_weight=0.5
)
```
---

## **검색 전략 결정 시 주의사항**

1. **검색 전략 선택**은 `비즈니스 요구사항`에 따라 결정
   - 균형 → Hybrid (Dense + Sparse BM25 - Kiwi)
   - 품질 최우선 → ColBERT 및 Multi-Vector 고려(근데, 저의 프로젝트 경험 상 대부분은 필요가 없었습니다.)

2. **한국어는 Kiwi 형태소 분석기를 꼭 쓴다고 생각해주세요.**
   - 한국어 오타 및 형태소 분석기를 통해 여러 의미를 내포하고 있더라도 정확하게 검색어 변형이 가능
   - 사용자 사전으로 도메인 용어 추가 가능
   - 불용어 제거로 검색 정확도 상승
   - 만약, Elastic Search(또는 OpenSearch) 를 쓴다면 Nori 형태소 분석기 를 무조건 쓰도록 하세요.

3. Embedding / Reranker 는 `최대한 로컬 모델로 운영`하시는게 비용 계산에 유리합니다.
   - Qwen3-Embedding-4B via `TEI`
   - Qwen3-Reranker-0.6B via `TEI`

4. **ColBERT는 품질 최우선 시**
   - Token-level 정밀도 → SOTA 성능
   - 다국어 Cross-lingual 검색 우수
   - [LFM2-ColBERT-350M](https://www.liquid.ai/blog/lfm2-colbert-350m-one-model-to-embed-them-all)
   - 그러나, Computing Cost 가 추가로 들어간다는 점은 항상 걸리는 포인트니 꼭 Test Level 에서 검증해보고 구현하시길 바랍니다.

5. **SPLADE는 차세대 Sparse Embedding 모델로 이해하시면 좋습니다.**
   - BM25의 속도 + Dense의 의미 이해
   - 아직은 Sentence Transformers 로만 Inference 가 가능하고 성능 검증이 더 필요하기 때문에 실전 프로젝트에서의 사용은 어렵습니다.
   - 다만, Sparse Embedding Model 에 대한 연구가 계속 되고 있는 만큼 이 키워드에 관심을 가지고 지켜보신다면 금방 상용화되지 않을까 싶습니다.
   - 한국어 모델: [yjoonjang/splade-ko-v1.0](https://huggingface.co/yjoonjang/splade-ko-v1.0)
